In [1]:
import pandas as pd
import wikipediaapi
from bs4 import BeautifulSoup
import requests

# Load the Excel file again to define the DataFrame
file_path = 'players.xlsx'  # Update the path
df = pd.read_excel(file_path)

In [6]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

# Load the Excel file
file_path = r'"C:\\Users\\Mihir\\8_CAPSTONE PROJECT\\tableau_vis\\players.xlsx"'  # Update this path if needed
df = pd.read_excel(file_path)

# Define a user agent string
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
}

# Function to search Wikipedia using the API
def search_wikipedia(player_name):
    search_url = f"https://en.wikipedia.org/w/api.php?action=query&list=search&srsearch={player_name}&format=json"
    
    response = requests.get(search_url, headers=headers)
    data = response.json()
    
    # Check if there are search results
    if len(data['query']['search']) > 0:
        # Get the title of the first result
        page_title = data['query']['search'][0]['title']
        page_url = f"https://en.wikipedia.org/wiki/{page_title.replace(' ', '_')}"
        return page_url
    else:
        return None

# Function to search for any images on the Wikipedia page (fallback)
def search_fallback_image(soup):
    images = soup.find_all('img')
    if images:
        for img in images:
            # Ensure we are getting a relevant image (not an icon)
            if 'thumb' in img.get('class', []) and 'src' in img.attrs:
                return 'https:' + img['src']
    return None

# Function to get the image link from the player's Wikipedia page
def get_wikipedia_image(player_name):
    try:
        # First, search for the Wikipedia page using the player name
        page_url = search_wikipedia(player_name)
        
        if page_url:
            print(f"Fetching page for: {player_name} - {page_url}")
            
            # Pause to avoid overwhelming Wikipedia
            time.sleep(1)
            
            # Make a request to the Wikipedia page
            player_response = requests.get(page_url, headers=headers)
            player_soup = BeautifulSoup(player_response.content, 'html.parser')
            
            # Try to find the infobox image (if available)
            infobox = player_soup.find('table', {'class': 'infobox'})
            if infobox:
                image = infobox.find('img')
                if image:
                    image_url = 'https:' + image['src']
                    return image_url
                else:
                    print(f"No image found in infobox for: {player_name}")
            else:
                print(f"No infobox found for: {player_name}")
            
            # Fallback: search for any other image on the page
            fallback_image = search_fallback_image(player_soup)
            if fallback_image:
                return fallback_image
            else:
                print(f"No fallback image found for: {player_name}")
                return None
        else:
            print(f"No search results for: {player_name}")
            return None
    except Exception as e:
        print(f"Error fetching image for {player_name}: {str(e)}")
        return None

# Apply the function to the list of players
df['Wikipedia Image Link'] = df['Names'].apply(get_wikipedia_image)

# Save the updated DataFrame with image links to a new Excel file
output_file = r'C:\\Users\\Mihir\\8_CAPSTONE PROJECT\\tableau_vis\\players_with_images.xlsx'
df.to_excel(output_file, index=False)

# Print the DataFrame to check results
print(df[['Names', 'Wikipedia Image Link']].head())

output_file  # Return the path to the saved file


Fetching page for: John Anthony Brooks - https://en.wikipedia.org/wiki/John_Brooks_(soccer,_born_1993)
Fetching page for: Emerson Hyndman - https://en.wikipedia.org/wiki/Emerson_Hyndman
Fetching page for: Lynden Gooch - https://en.wikipedia.org/wiki/Lynden_Gooch
Fetching page for: Timothy Weah - https://en.wikipedia.org/wiki/Timothy_Weah
Fetching page for: Djordje Mihailovic - https://en.wikipedia.org/wiki/Djordje_Mihailovic
Fetching page for: Jack de Vries - https://en.wikipedia.org/wiki/Jack_de_Vries
No infobox found for: Jack de Vries
No fallback image found for: Jack de Vries
Fetching page for: Kahveh Zahiroleslam - https://en.wikipedia.org/wiki/Kahveh_Zahiroleslam
No image found in infobox for: Kahveh Zahiroleslam
No fallback image found for: Kahveh Zahiroleslam
Fetching page for: Bobby Wood - https://en.wikipedia.org/wiki/Bobby_Wood
Fetching page for: Eugene Starikov - https://en.wikipedia.org/wiki/Eugene_Starikov
Fetching page for: Dillon Powers - https://en.wikipedia.org/wiki/D

'C:\\\\Users\\\\Mihir\\\\Desktop\\\\players_with_images.xlsx'